# Import Packages

In [1]:
from scipy.io import loadmat
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from tensorflow.keras import backend as K
import tensorflow as tf
import tarfile

from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms

from io import BytesIO
from PIL import Image
import gzip
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import data_handler as dh
import utility_handler as uh
import backbond_handler as bh

r_seed = 0

# To load known-unknown class setting as the same as
# The details can be found here:
# https://github.com/iCGY96/ARPL/blob/master/split.py
num_shuffle = len(uh.splits_2020['cifar100-50'])
data_load_func = dh.load_cifar10
data_load_func_plus = dh.load_cifar100
plus = 50

backbone = bh.VGG1132
device = 'cuda' if torch.cuda.is_available() else 'cpu'
epoch_max = 120

# Loading Data

In [2]:
trX, trY, teX, teY = data_load_func()
_, _, teX_plus, teY_plus = data_load_func_plus()

# Encoding

In [3]:
def split_know_unkn_cifar_plus(trX,
                               trY,
                               teX,
                               teY,
                               teX_plus,
                               teY_plus,
                               plus,
                               num_shuffle=5,
                               know_class_list=None,
                               unkn_class_list=None,
                               r_seed=0):
    np.random.seed(r_seed)

    know_trX_list = []
    know_trY_list = []
    know_teX_list = []
    know_teY_list = []
    unkn_teX_list = []
    unkn_teY_list = []

    know_class_list = uh.splits_2020['cifar100']
    unkn_class_list = uh.splits_2020['cifar100-50']

    print(f'numpy random seed {r_seed}')

    for i in range(num_shuffle):

        know_trX_list_temp = []
        know_trY_list_temp = []
        know_teX_list_temp = []
        know_teY_list_temp = []
        unkn_teX_list_temp = []
        unkn_teY_list_temp = []

        print(f'Ranodm set {i}')
        print(f'    know classes {know_class_list[i]}')
        print(f'    unkn classes {unkn_class_list[i]}')

        for c_label in know_class_list[i]:

            trID = np.where(trY == c_label)
            teID = np.where(teY == c_label)
            trX_new = trX[trID]
            trY_new = trY[trID]
            teX_new = teX[teID]
            teY_new = teY[teID]
            know_trX_list_temp.append(trX_new)
            know_trY_list_temp.append(trY_new)
            know_teX_list_temp.append(teX_new)
            know_teY_list_temp.append(teY_new)

        for c_label in unkn_class_list[i]:
            teID = np.where(teY_plus == c_label)
            teX_new = teX_plus[teID]
            teY_new = teY_plus[teID]
            unkn_teX_list_temp.append(teX_new)
            unkn_teY_list_temp.append(teY_new)

        know_trX_list.append(np.concatenate(know_trX_list_temp, axis=0))
        know_trY_list.append(np.concatenate(know_trY_list_temp, axis=0))
        know_teX_list.append(np.concatenate(know_teX_list_temp, axis=0))
        know_teY_list.append(np.concatenate(know_teY_list_temp, axis=0))
        unkn_teX_list.append(np.concatenate(unkn_teX_list_temp, axis=0))
        unkn_teY_list.append(np.concatenate(unkn_teY_list_temp, axis=0))

    dataset_X_dict = {
        'kn_tr': know_trX_list,
        'kn_te': know_teX_list,
        'uk_te': unkn_teX_list
    }
    dataset_Y_dict = {
        'kn_tr': know_trY_list,
        'kn_te': know_teY_list,
        'uk_te': unkn_teY_list
    }

    return dataset_X_dict, dataset_Y_dict

In [4]:
# ====================================================== #
# random split 10 classes into 6 known 4 unknown classes #
# ====================================================== #
dataset_X_dict, dataset_Y_dict = split_know_unkn_cifar_plus(
    trX,
    trY,
    teX,
    teY,
    teX_plus,
    teY_plus,
    plus,
    num_shuffle=num_shuffle,
    know_class_list=None,
    unkn_class_list=None,
    r_seed=r_seed)

# ============= #
teX_plus,
# build encoder #
# ============= #
encoder_list = bh.build_encoder(dataset_X_dict, dataset_Y_dict, backbone,
                                device, r_seed, epoch_max,
                                data_load_func.__name__)

# ============= #
# encode images #
# ============= #
kn_trX_encode_list = []
kn_teX_encode_list = []
uk_teX_encode_list = []

# shuffle is enabled during traing
# therefore, we need to store the label at encoding stage
kn_trY_encode_list = []
kn_teY_encode_list = []
uk_teY_encode_list = []

for idx, encoder in enumerate(encoder_list):
    kn_tr_Dataset = bh.image_data_to_dataloader(dataset_X_dict['kn_tr'][idx],
                                                dataset_Y_dict['kn_tr'][idx],
                                                128)
    kn_te_Dataset = bh.image_data_to_dataloader(dataset_X_dict['kn_te'][idx],
                                                dataset_Y_dict['kn_te'][idx],
                                                128)
    uk_te_Dataset = bh.image_data_to_dataloader(dataset_X_dict['uk_te'][idx],
                                                dataset_Y_dict['uk_te'][idx],
                                                128)

    kn_tr_encode_XY = bh.encoding(kn_tr_Dataset, encoder, device)
    kn_te_encode_XY = bh.encoding(kn_te_Dataset, encoder, device)
    uk_te_encode_XY = bh.encoding(uk_te_Dataset, encoder, device)

    kn_trX_encode_list.append(kn_tr_encode_XY[:, :-1])
    kn_teX_encode_list.append(kn_te_encode_XY[:, :-1])
    uk_teX_encode_list.append(uk_te_encode_XY[:, :-1])

    kn_trY_encode_list.append(kn_tr_encode_XY[:, -1])
    kn_teY_encode_list.append(kn_te_encode_XY[:, -1])
    uk_teY_encode_list.append(uk_te_encode_XY[:, -1])

numpy random seed 0
Ranodm set 0
    know classes [4, 7, 9, 1]
    unkn classes [27, 94, 29, 77, 88, 26, 69, 48, 75, 5, 59, 93, 39, 57, 45, 40, 78, 20, 98, 47, 66, 70, 91, 76, 41, 83, 99, 32, 53, 72, 2, 95, 21, 73, 84, 68, 35, 11, 55, 60, 30, 25, 1, 9, 8, 0, 46, 52, 49, 71]
Ranodm set 1
    know classes [6, 7, 1, 9]
    unkn classes [65, 97, 86, 24, 45, 67, 2, 3, 91, 98, 79, 29, 62, 82, 33, 76, 0, 35, 5, 16, 54, 11, 99, 52, 85, 1, 25, 66, 28, 84, 23, 56, 75, 46, 21, 72, 55, 68, 8, 69, 41, 9, 49, 40, 73, 60, 48, 30, 95, 71]
Ranodm set 2
    know classes [9, 6, 1, 7]
    unkn classes [20, 83, 65, 97, 94, 2, 93, 16, 67, 29, 62, 33, 24, 98, 5, 86, 35, 54, 0, 91, 52, 66, 85, 84, 56, 11, 1, 76, 25, 55, 21, 99, 72, 41, 23, 75, 28, 68, 69, 46, 8, 9, 49, 40, 73, 60, 48, 95, 30, 71]
Ranodm set 3
    know classes [6, 4, 9, 1]
    unkn classes [92, 82, 77, 64, 5, 33, 62, 56, 70, 0, 20, 28, 67, 14, 84, 53, 91, 29, 85, 2, 52, 83, 75, 35, 11, 21, 72, 98, 55, 1, 41, 76, 25, 66, 69, 9, 48, 54, 40, 23, 

# Open-set Enrichment

In [5]:
from aosr_utility import sample_enrichment_IF

kn_trX_encode_zqX_list = []
kn_trX_encode_zqW_list = []

for i in range(num_shuffle):
    encX = kn_trX_encode_list[i]
    zqX, zqW = sample_enrichment_IF(0, encX, encX.shape[0] * 3)
    zqW = 1 - zqW * 1.0

    tau = np.sort(zqW)[int(zqW.shape[0] * .001)]

    # Unknown Samples
    cond1 = (zqW > tau)
    cond1 = np.squeeze(cond1)
    # Known Samples
    cond2 = (zqW <= tau)
    cond2 = np.squeeze(cond2)
    # Wipe Known Samples
    zqW[cond2] = 0
    # Balance the weights
    zqW = zqW * (zqW.shape[0] / zqW.sum())

    kn_trX_encode_zqX_list.append(zqX)
    kn_trX_encode_zqW_list.append(zqW)

(48437,)
(46208,)
(46063,)
(48630,)
(53621,)


# Open-set Learning

In [6]:
from aosr_utility import aosr_risk, EarlyStoppingBeforeOverfit
from sklearn.preprocessing import LabelEncoder

for i in range(num_shuffle):
    le = LabelEncoder()
    kn_trY_encode_list[i] = le.fit_transform(kn_trY_encode_list[i])
    kn_teY_encode_list[i] = le.transform(kn_teY_encode_list[i])


def build_openset_learner(zpX, zpY, zqX, zqW, mu, num_class):

    bh.set_random_seed(r_seed, device)

    detector = tf.keras.models.Sequential([
        tf.keras.layers.Dense(num_class + 1),
        tf.keras.layers.Activation(activation='softmax')
    ])

    detector.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])
    detector.fit(zpX, zpY * 1.0, epochs=2)

    detector.compile(optimizer='adam',
                     loss=aosr_risk(detector, zqX, zqW, zpX, mu, num_class),
                     metrics=['accuracy'])

    K.set_value(detector.optimizer.learning_rate, 0.001)
    detector.fit(zpX,
                 zpY,
                 epochs=100,
                 callbacks=[EarlyStoppingBeforeOverfit()])
    return detector


mu = 0.005
num_class = np.unique(kn_trY_encode_list[0]).shape[0]
print(mu, num_class)
detector_list = []

for i in range(num_shuffle):

    zpX = kn_trX_encode_list[i]
    zpY = kn_trY_encode_list[i]
    zqX = kn_trX_encode_zqX_list[i]
    zqW = kn_trX_encode_zqW_list[i]
    detector = build_openset_learner(zpX, zpY, zqX, zqW, mu, num_class)
    detector_list.append(detector)

0.005 4
Epoch 1/2
625/625 [==============================] - 1s 780us/step - loss: 2.4410 - accuracy: 0.6021
Epoch 2/2
625/625 [==============================] - 0s 781us/step - loss: 0.0283 - accuracy: 1.0000
Epoch 1/100
625/625 [==============================] - 1s 1ms/step - loss: 78850220.2045 - accuracy: 0.9979
Epoch 2/100
625/625 [==============================] - 1s 1ms/step - loss: 22.2762 - accuracy: 0.9963
Epoch 3/100
625/625 [==============================] - 1s 1ms/step - loss: 22.2726 - accuracy: 0.9962
Epoch 4/100
625/625 [==============================] - 1s 1ms/step - loss: 22.2754 - accuracy: 0.9973
Epoch 5/100
625/625 [==============================] - 1s 1ms/step - loss: 22.2755 - accuracy: 0.9969
Epoch 6/100
625/625 [==============================] - 1s 1ms/step - loss: 22.2734 - accuracy: 0.9967
Epoch 7/100
625/625 [==============================] - 1s 1ms/step - loss: 22.2731 - accuracy: 0.9970
Epoch 8/100
625/625 [==============================] - 1s 1ms/step - l

# Evaluation

In [7]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import f1_score

In [8]:
actY_list = []
preY_list = []

actY_know_list = []
preY_know_list = []

actY_binary_list = []
preY_binary_list = []

for i in range(num_shuffle):
    
    actY = np.zeros(kn_teX_encode_list[i].shape[0] + uk_teX_encode_list[i].shape[0])
    actY[:kn_teX_encode_list[i].shape[0]] = kn_teY_encode_list[i]
    actY[kn_teX_encode_list[i].shape[0]:] = num_class
    
    kn_preY = detector_list[i].predict(kn_teX_encode_list[i])
    uk_preY = detector_list[i].predict(uk_teX_encode_list[i])
    preY = np.concatenate([kn_preY, uk_preY], axis=0)
    
    actY_know_list.append(kn_teY_encode_list[i])
    preY_know_list.append(kn_preY.argmax(axis=1))
    
    actY_list.append(actY)
    preY_list.append(preY)
    
    actY_bi = np.array(actY)
    preY_bi = preY[:, -1]
    
    actY_bi[actY_bi<4] = 0
    actY_bi[actY_bi==4] = 1
    
    actY_binary_list.append(actY_bi)
    preY_binary_list.append(preY_bi)

In [9]:
acc_list = []
auc_bi_list = []
auc_list = []
f1_list = []
for i in range(num_shuffle):
    print(f'shuffle {i}')
    print('    acc score', accuracy_score(actY_know_list[i], preY_know_list[i]))
    print('    acc known', accuracy_score(actY_know_list[i][actY_know_list[i]==0], preY_know_list[i][actY_know_list[i]==0]))
    print('    acc uknow', accuracy_score(actY_know_list[i][actY_know_list[i]==1], preY_know_list[i][actY_know_list[i]==1]))
    acc_list.append(roc_auc_score(actY_list[i], preY_list[i], multi_class='ovr', average='weighted'))
    auc_bi_list.append(roc_auc_score(actY_binary_list[i], preY_binary_list[i]))
    fpr, tpr, thresholds = metrics.roc_curve(actY_binary_list[i], preY_binary_list[i], pos_label=1)
    auc_list.append(metrics.auc(fpr, tpr))
    f1_list.append(f1_score(actY_list[i], preY_list[i].argmax(1), average='macro'))
    print('    f1 macro score', f1_list[i])
    print('    auc only score', auc_list[i])
    print('    auc multi-score', acc_list[i])
    print('    auc binary score', auc_bi_list[i])
print(f'Avg F1 {np.mean(f1_list)}, {np.std(f1_list)}')
print(f'Avg AUC {np.mean(auc_list)}, {np.std(auc_list)}')
print(f'Avg Accurac {np.mean(acc_list)}, {np.std(acc_list)}')
print(f'Avg ROC_AUC {np.mean(auc_bi_list)}, {np.std(auc_bi_list)}')

shuffle 0
    acc score 0.929
    acc known 0.949
    acc uknow 0.951
    f1 macro score 0.569453725170306
shuffle 1
    acc score 0.876
    acc known 0.843
    acc uknow 0.923
    f1 macro score 0.8021300472697414
shuffle 2
    acc score 0.876
    acc known 0.843
    acc uknow 0.923
    f1 macro score 0.8030578876530724
shuffle 3
    acc score 0.88075
    acc known 0.892
    acc uknow 0.918
    f1 macro score 0.7400894382075682
shuffle 4
    acc score 0.901
    acc known 0.902
    acc uknow 0.917
    f1 macro score 0.6154330744752758
Avg F1 0.7060328345551927, 0.09661320522515948
Avg Accurac 0.8881658930555554, 0.0506909394562439
